# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 17 2022 2:54PM,237439,19,ACG-2102490103,ACG North America LLC,Released
1,May 17 2022 2:54PM,237439,19,ACG-2102490104,ACG North America LLC,Released
2,May 17 2022 2:51PM,237438,19,ADV80002718,"AdvaGen Pharma, Ltd",Released
3,May 17 2022 2:47PM,237437,20,8027215,Reliable 1 Laboratories LLC,Released
4,May 17 2022 2:47PM,237437,20,8027837,Reliable 1 Laboratories LLC,Released
5,May 17 2022 2:47PM,237437,20,8028429,Reliable 1 Laboratories LLC,Released
6,May 17 2022 2:47PM,237437,20,8028430,Reliable 1 Laboratories LLC,Released
7,May 17 2022 2:47PM,237437,20,8028431,Reliable 1 Laboratories LLC,Released
8,May 17 2022 2:47PM,237437,20,8030536,Reliable 1 Laboratories LLC,Released
9,May 17 2022 2:44PM,237436,19,DEX0006805,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,237436,Executing,5
49,237436,Released,8
50,237437,Released,6
51,237438,Released,1
52,237439,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237433,NaN,NaN,3.0
237436,NaN,5.0,8.0
237437,NaN,NaN,6.0
237438,NaN,NaN,1.0
237439,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0.0,1.0,0.0
237336,0.0,0.0,1.0
237342,0.0,1.0,0.0
237349,0.0,0.0,1.0
237350,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0,1,0
237336,0,0,1
237342,0,1,0
237349,0,0,1
237350,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,0,1,0
1,237336,0,0,1
2,237342,0,1,0
3,237349,0,0,1
4,237350,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,,1,
1,237336,,,1
2,237342,,1,
3,237349,,,1
4,237350,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 17 2022 2:54PM,237439,19,ACG North America LLC
2,May 17 2022 2:51PM,237438,19,"AdvaGen Pharma, Ltd"
3,May 17 2022 2:47PM,237437,20,Reliable 1 Laboratories LLC
9,May 17 2022 2:44PM,237436,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
22,May 17 2022 2:41PM,237433,18,Hush Hush
25,May 17 2022 2:33PM,237432,10,ISDIN Corporation
26,May 17 2022 2:28PM,237431,10,ISDIN Corporation
54,May 17 2022 2:26PM,237430,20,Sartorius Stedim North America
55,May 17 2022 2:25PM,237429,10,ISDIN Corporation
56,May 17 2022 2:22PM,237428,20,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 17 2022 2:54PM,237439,19,ACG North America LLC,,,2
1,May 17 2022 2:51PM,237438,19,"AdvaGen Pharma, Ltd",,,1
2,May 17 2022 2:47PM,237437,20,Reliable 1 Laboratories LLC,,,6
3,May 17 2022 2:44PM,237436,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,5,8
4,May 17 2022 2:41PM,237433,18,Hush Hush,,,3
5,May 17 2022 2:33PM,237432,10,ISDIN Corporation,,,1
6,May 17 2022 2:28PM,237431,10,ISDIN Corporation,,,28
7,May 17 2022 2:26PM,237430,20,Sartorius Stedim North America,,,1
8,May 17 2022 2:25PM,237429,10,ISDIN Corporation,,,1
9,May 17 2022 2:22PM,237428,20,Sartorius Stedim North America,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 2:54PM,237439,19,ACG North America LLC,2,,
1,May 17 2022 2:51PM,237438,19,"AdvaGen Pharma, Ltd",1,,
2,May 17 2022 2:47PM,237437,20,Reliable 1 Laboratories LLC,6,,
3,May 17 2022 2:44PM,237436,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8,5,
4,May 17 2022 2:41PM,237433,18,Hush Hush,3,,
5,May 17 2022 2:33PM,237432,10,ISDIN Corporation,1,,
6,May 17 2022 2:28PM,237431,10,ISDIN Corporation,28,,
7,May 17 2022 2:26PM,237430,20,Sartorius Stedim North America,1,,
8,May 17 2022 2:25PM,237429,10,ISDIN Corporation,1,,
9,May 17 2022 2:22PM,237428,20,Sartorius Stedim North America,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 2:54PM,237439,19,ACG North America LLC,2,,
1,May 17 2022 2:51PM,237438,19,"AdvaGen Pharma, Ltd",1,,
2,May 17 2022 2:47PM,237437,20,Reliable 1 Laboratories LLC,6,,
3,May 17 2022 2:44PM,237436,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8,5,
4,May 17 2022 2:41PM,237433,18,Hush Hush,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 2:54PM,237439,19,ACG North America LLC,2.0,NaN,NaN
1,May 17 2022 2:51PM,237438,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
2,May 17 2022 2:47PM,237437,20,Reliable 1 Laboratories LLC,6.0,NaN,NaN
3,May 17 2022 2:44PM,237436,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8.0,5.0,NaN
4,May 17 2022 2:41PM,237433,18,Hush Hush,3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 2:54PM,237439,19,ACG North America LLC,2.0,0.0,0.0
1,May 17 2022 2:51PM,237438,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
2,May 17 2022 2:47PM,237437,20,Reliable 1 Laboratories LLC,6.0,0.0,0.0
3,May 17 2022 2:44PM,237436,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8.0,5.0,0.0
4,May 17 2022 2:41PM,237433,18,Hush Hush,3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3086303,103.0,69.0,0.0
12,237342,0.0,1.0,0.0
15,237363,6.0,4.0,5.0
16,949472,4.0,0.0,0.0
18,474456,3.0,1.0,0.0
19,1661992,18.0,6.0,0.0
20,3323712,74.0,3.0,0.0
21,237381,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3086303,103.0,69.0,0.0
1,12,237342,0.0,1.0,0.0
2,15,237363,6.0,4.0,5.0
3,16,949472,4.0,0.0,0.0
4,18,474456,3.0,1.0,0.0
5,19,1661992,18.0,6.0,0.0
6,20,3323712,74.0,3.0,0.0
7,21,237381,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,103.0,69.0,0.0
1,12,0.0,1.0,0.0
2,15,6.0,4.0,5.0
3,16,4.0,0.0,0.0
4,18,3.0,1.0,0.0
5,19,18.0,6.0,0.0
6,20,74.0,3.0,0.0
7,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,103.0
1,12,Released,0.0
2,15,Released,6.0
3,16,Released,4.0
4,18,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
Executing,69.0,1.0,4.0,0.0,1.0,6.0,3.0,1.0
Released,103.0,0.0,6.0,4.0,3.0,18.0,74.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
1,Executing,69.0,1.0,4.0,0.0,1.0,6.0,3.0,1.0
2,Released,103.0,0.0,6.0,4.0,3.0,18.0,74.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
1,Executing,69.0,1.0,4.0,0.0,1.0,6.0,3.0,1.0
2,Released,103.0,0.0,6.0,4.0,3.0,18.0,74.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()